In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import jax

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Couplings J1 and J2
J = [1, 0.2]
L = 8

In [4]:
# Define custom graph
edge_colors = []
for i in range(L):
    edge_colors.append([i, (i+1)%L, 1])
    edge_colors.append([i, (i+2)%L, 2])
# Define the netket graph object
g = nk.graph.Graph(edges=edge_colors)

In [5]:
#Sigma^z*Sigma^z interactions
sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz))
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])

bond_operator = [
    (J[0] * mszsz).tolist(),
    (J[1] * mszsz).tolist(),
    (-J[0] * exchange).tolist(),  
    (J[1] * exchange).tolist(),
]

bond_color = [1, 2, 1, 2]

In [6]:
hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)

In [7]:
op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)

In [8]:
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp

class FFNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=2*x.shape[-1], 
                     use_bias=True, 
                     param_dtype=np.complex128, 
                     kernel_init=nn.initializers.normal(stddev=0.01), 
                     bias_init=nn.initializers.normal(stddev=0.01)
                    )(x)
        x = nknn.log_cosh(x)
        x = jnp.sum(x, axis=-1)
        return x
        
    def get_kernel(self, variables):
        if 'params' in variables:
            return variables['params']['Dense_0']['kernel']
        else:
            return variables['Dense_0']['kernel']
    
    def get_bias(self, variables):
        if 'params' in variables:
            return variables['params']['Dense_0']['bias']
        else:
            return variables['Dense_0']['bias']

    
model = FFNN()

In [9]:
sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g, d_max = 2)
vs = nk.vqs.MCState(sa, model, n_samples=1008)
opt = nk.optimizer.Sgd(learning_rate=0.01)
sr = nk.optimizer.SR(diag_shift=0.01)
gs = nk.VMC(hamiltonian=op, optimizer=opt, variational_state=vs, preconditioner=sr)

In [10]:
vs

MCState(
  hilbert = Spin(s=1/2, total_sz=0.0, N=8),
  sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>),
  n_samples = 1008,
  n_discard_per_chain = 100,
  sampler_state = MetropolisSamplerState(rng state=[2176786589  232078714]),
  n_parameters = 144)

In [11]:
vs.parameters['Dense_0']['bias']

Array([ 1.00304571e-03-0.00462823j,  1.44231366e-03+0.00080073j,
        4.17539183e-03-0.002437j  , -3.16163406e-04-0.00411301j,
        1.42829848e-02+0.00376391j,  2.63412019e-03-0.00869977j,
       -2.47072662e-03-0.01098437j,  1.32942582e-02-0.00435606j,
       -8.36903787e-03-0.00016218j,  1.00409652e-02-0.00533245j,
        3.14430449e-03-0.00679536j,  1.81901829e-03-0.00137474j,
        1.96984670e-03+0.00605823j, -5.13388500e-03+0.00893002j,
        8.98758738e-06+0.0066975j ,  3.37528310e-03-0.00639027j],      dtype=complex128)

In [12]:
# Define a função de perda (loss function) para o treinamento
def loss(params, structure_factor):
    output = model.apply({'params': params}, structure_factor)
    return jnp.mean(output)

In [13]:
# Inicializa os parâmetros treináveis
key = jax.random.PRNGKey(0)
params = model.init(key, jnp.ones((1, 10)))

In [14]:
sf = []
sites = []
structure_factor = nk.operator.LocalOperator(hi, dtype=complex)
for i in range(0, L):
    for j in range(0, L):
        structure_factor += (nk.operator.spin.sigmaz(hi, i)*nk.operator.spin.sigmaz(hi, j))*((-1)**(i-j))/L

In [15]:
#Função para acessar o bias e o kernel após o treinamento
def get_model_params(trained_params):
    kernel = model.get_kernel(trained_params)
    bias = model.get_bias(trained_params)
    return kernel, bias

In [53]:
# Define uma lista para armazenar os parâmetros do modelo durante o treinamento
trained_params_list = []
# Função de callback para salvar os parâmetros durante o treinamento
def save_params(step, params, energy):
    #print("############ step ", step)
    #print("############ param", params)
    print("############ it", step, )
    print("############ energy", energy.state.parameters)
    
    trained_params_list.append(params.copy())
    return True

In [54]:
gs.run(out='test', n_iter=100, obs={'Structure Factor': structure_factor}, callback=save_params)

  2%|█▍                                                                        | 2/100 [00:00<00:06, 15.77it/s, Energy=-13.402715-0.000055j ± 0.000080 [σ²=0.000006, R̂=1.0044]]

############ it 900
############ energy {'Dense_0': {'bias': Array([-4.82841435e-05-2.29073767e-04j,  8.65871835e-03-2.85838900e-03j,
       -4.21049469e-04-2.67483231e-04j, -3.38149388e-03-5.69133036e-04j,
        3.46351402e-03+1.04508464e-03j,  1.28018531e-03-6.95217792e-04j,
        1.12041017e-03+7.67753945e-05j,  2.66575171e-03-1.05763610e-03j,
        2.15861551e-04+3.95056108e-04j,  2.53202820e-04-3.83959483e-04j,
        1.12043007e-03+9.25653680e-04j,  1.17077368e-04+3.63169894e-04j,
       -4.29518897e-04+4.78189762e-04j, -4.08358626e-04+7.11630426e-04j,
        9.70409993e-04+3.99337248e-04j, -1.75247854e-03-2.08786381e-03j],      dtype=complex128), 'kernel': Array([[ 1.36685063e-01+0.03507771j, -9.51300469e-03+0.02253311j,
         7.31329585e-02-0.08181091j,  1.55009769e-02+0.08672608j,
        -8.70949314e-02-0.13354672j,  1.64291939e-02+0.18974254j,
         1.28864800e-01-0.02193877j,  4.63882315e-02-0.01199322j,
        -8.39113218e-02-0.02271157j, -5.49566663e-02-0.1

  6%|████▍                                                                     | 6/100 [00:00<00:06, 15.24it/s, Energy=-13.402785-0.000026j ± 0.000085 [σ²=0.000007, R̂=1.0065]]

############ it 904
############ energy {'Dense_0': {'bias': Array([-5.40975533e-05-2.24035985e-04j,  8.65647290e-03-2.85650287e-03j,
       -4.27899567e-04-2.65068299e-04j, -3.37027629e-03-5.61975380e-04j,
        3.44228262e-03+1.04581744e-03j,  1.27844086e-03-6.99123548e-04j,
        1.11701606e-03+6.64477991e-05j,  2.67114854e-03-1.04800953e-03j,
        2.19089946e-04+3.87766686e-04j,  2.49976733e-04-3.78779665e-04j,
        1.10602081e-03+9.22051499e-04j,  1.11937471e-04+3.59365167e-04j,
       -4.23377254e-04+4.78409793e-04j, -4.07504203e-04+6.99707913e-04j,
        9.71266474e-04+3.83852511e-04j, -1.74776499e-03-2.07624359e-03j],      dtype=complex128), 'kernel': Array([[ 1.36711184e-01+0.03508498j, -9.51050762e-03+0.02252728j,
         7.31482988e-02-0.08180673j,  1.54860326e-02+0.08671875j,
        -8.71010721e-02-0.13354769j,  1.64250427e-02+0.18971583j,
         1.28867595e-01-0.02193925j,  4.63775347e-02-0.01198093j,
        -8.39287372e-02-0.02267505j, -5.49600894e-02-0.1

 10%|███████▎                                                                 | 10/100 [00:00<00:05, 15.09it/s, Energy=-13.402862+0.000003j ± 0.000069 [σ²=0.000005, R̂=1.0019]]

...

 96%|█████████████████████████████████████████████████████████████████████████▉   | 96/100 [00:06<00:00, 15.21it/s, Energy=-13.40276-0.00006j ± 0.00011 [σ²=0.00001, R̂=1.0044]]

############ it 994
############ energy {'Dense_0': {'bias': Array([ 1.40065771e-04-0.0002789j ,  8.32847020e-03-0.0026847j ,
       -4.84363004e-04-0.00043263j, -2.98918482e-03-0.00038383j,
        2.91922540e-03+0.00108249j,  8.33391525e-04-0.00068839j,
        8.69892010e-04+0.00041021j,  3.07744470e-03-0.00164805j,
        5.09300660e-04+0.00046539j,  1.04577255e-03-0.00089467j,
        9.43057040e-04+0.00074444j, -1.73061296e-05+0.0004996j ,
       -3.03667789e-04+0.00059805j, -8.82614784e-04+0.00062071j,
        2.89926369e-04+0.00046098j, -2.00614081e-03-0.00186515j],      dtype=complex128), 'kernel': Array([[ 1.37197238e-01+0.03463857j, -9.29617553e-03+0.02264911j,
         7.32723118e-02-0.0823976j ,  1.52189835e-02+0.08787359j,
        -8.69734222e-02-0.13410465j,  1.67850803e-02+0.19050851j,
         1.29414001e-01-0.02240875j,  4.56970167e-02-0.01184038j,
        -8.46643783e-02-0.02334125j, -5.63531684e-02-0.16722031j,
         2.81815588e-02+0.24411313j,  1.95625568e-02-0

100%|████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.06it/s, Energy=-13.40290+0.00006j ± 0.00012 [σ²=0.00001, R̂=1.0039]]

############ it 997
############ energy {'Dense_0': {'bias': Array([ 1.26930213e-04-0.00026616j,  8.32112920e-03-0.00269223j,
       -4.81845917e-04-0.0004198j , -2.98840979e-03-0.00039401j,
        2.90687743e-03+0.00107574j,  8.33827536e-04-0.00069402j,
        8.80985377e-04+0.00038935j,  3.07204135e-03-0.00161444j,
        5.01445208e-04+0.00045781j,  1.03064838e-03-0.00086355j,
        9.52572688e-04+0.00073899j, -1.81931529e-05+0.00045249j,
       -3.01895966e-04+0.00059471j, -8.78109562e-04+0.00061044j,
        2.99363445e-04+0.00045193j, -2.00183794e-03-0.00186158j],      dtype=complex128), 'kernel': Array([[ 1.37231059e-01+0.03463922j, -9.29441417e-03+0.02263626j,
         7.33055787e-02-0.08240055j,  1.51901058e-02+0.08786446j,
        -8.69872101e-02-0.13410018j,  1.67720285e-02+0.19048571j,
         1.29423355e-01-0.02241411j,  4.56902785e-02-0.01181918j,
        -8.46789656e-02-0.02329776j, -5.63704924e-02-0.1672304j ,
         2.81608343e-02+0.24409116j,  1.95477589e-02-0

(JsonLog('test', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.024415254592895508
   	Params: 0.0006890296936035156,)

vs.parameters

In [45]:
vs

MCState(
  hilbert = Spin(s=1/2, total_sz=0.0, N=8),
  sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>),
  n_samples = 1008,
  n_discard_per_chain = 100,
  sampler_state = MetropolisSamplerState(# accepted = 13054/20864 (62.56710122699386%), rng state=[ 924178737 3718882349]),
  n_parameters = 144)

In [22]:
def save_params(step, params, energy):
    trained_params_list.append(params.copy())

# Executa o treinamento com múltiplos ciclos
n_cycles = 10  # Número de ciclos de treinamento desejado
for _ in range(n_cycles):
    gs.run(out='test', n_iter=100, obs={'Structure Factor': structure_factor}, callback=save_params)


  0%|                                                                                                  | 0/100 [00:00<?, ?it/s, Energy=-8.54+0.00j ± 0.18 [σ²=31.64, R̂=1.0091]]


In [34]:
gs.run(out='test', n_iter=100, obs={'Structure Factor': structure_factor}, callback=save_params)

  2%|█▋                                                                                | 2/100 [00:00<00:06, 16.17it/s, Energy=-13.4016-0.0002j ± 0.0014 [σ²=0.0021, R̂=1.0069]]

############ step  100
############ param {'Structure Factor': 3.45+0.00j ± 0.10 [σ²=10.11, R̂=1.0615], 'Energy': -13.4013-0.0003j ± 0.0015 [σ²=0.0023, R̂=1.0075]}
############ energy Vmc(
  step_count = 100,
  state = MCState(hilbert = Spin(s=1/2, total_sz=0.0, N=8), sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>), n_samples = 1008))
############ step  101
############ param {'Structure Factor': 3.169+0.000j ± 0.096 [σ²=9.345, R̂=1.0522], 'Energy': -13.4039+0.0020j ± 0.0019 [σ²=0.0037, R̂=1.0025]}
############ energy Vmc(
  step_count = 101,
  state = MCState(hilbert = Spin(s=1/2, total_sz=0.0, N=8), sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>), n_samples = 1008))
############ step  102
############ param {'Structure Factor': 3.48+0.00j ± 0.10 [σ²=10.22, R

  6%|████▉                                                                             | 6/100 [00:00<00:06, 15.42it/s, Energy=-13.4019-0.0002j ± 0.0015 [σ²=0.0022, R̂=1.0065]]

############ step  104
############ param {'Structure Factor': 3.349+0.000j ± 0.100 [σ²=10.005, R̂=1.0389], 'Energy': -13.4030+0.0002j ± 0.0017 [σ²=0.0028, R̂=1.0111]}
############ energy Vmc(
  step_count = 104,
  state = MCState(hilbert = Spin(s=1/2, total_sz=0.0, N=8), sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>), n_samples = 1008))
############ step  105
############ param {'Structure Factor': 2.972+0.000j ± 0.095 [σ²=9.110, R̂=1.0757], 'Energy': -13.4025-0.0002j ± 0.0014 [σ²=0.0020, R̂=1.0075]}
############ energy Vmc(
  step_count = 105,
  state = MCState(hilbert = Spin(s=1/2, total_sz=0.0, N=8), sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>), n_samples = 1008))
############ step  106
############ param {'Structure Factor': 3.40+0.00j ± 0.10 [σ²=10.1

 10%|████████                                                                         | 10/100 [00:00<00:05, 15.09it/s, Energy=-13.4027-0.0004j ± 0.0017 [σ²=0.0029, R̂=1.0142]]

...

 96%|█████████████████████████████████████████████████████████████████████████████▊   | 96/100 [00:06<00:00, 15.62it/s, Energy=-13.4036+0.0005j ± 0.0010 [σ²=0.0010, R̂=1.0084]]

############ step  193
############ param {'Structure Factor': 3.52+0.00j ± 0.10 [σ²=10.43, R̂=1.0420], 'Energy': -13.4033-0.0004j ± 0.0010 [σ²=0.0010, R̂=1.0066]}
############ energy Vmc(
  step_count = 193,
  state = MCState(hilbert = Spin(s=1/2, total_sz=0.0, N=8), sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>), n_samples = 1008))
############ step  194
############ param {'Structure Factor': 3.123+0.000j ± 0.098 [σ²=9.588, R̂=1.0523], 'Energy': -13.40304+0.00017j ± 0.00089 [σ²=0.00079, R̂=1.0104]}
############ energy Vmc(
  step_count = 194,
  state = MCState(hilbert = Spin(s=1/2, total_sz=0.0, N=8), sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>), n_samples = 1008))
############ step  195
############ param {'Structure Factor': 3.28+0.00j ± 0.10 [σ²=10.2

100%|████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.47it/s, Energy=-13.40233-0.00050j ± 0.00067 [σ²=0.00045, R̂=1.0083]]

############ step  197
############ param {'Structure Factor': 3.115+0.000j ± 0.098 [σ²=9.622, R̂=1.0282], 'Energy': -13.40298+0.00012j ± 0.00084 [σ²=0.00072, R̂=1.0067]}
############ energy Vmc(
  step_count = 197,
  state = MCState(hilbert = Spin(s=1/2, total_sz=0.0, N=8), sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>), n_samples = 1008))
############ step  198
############ param {'Structure Factor': 3.236+0.000j ± 0.099 [σ²=9.857, R̂=1.0513], 'Energy': -13.40238-0.00011j ± 0.00085 [σ²=0.00073, R̂=1.0056]}
############ energy Vmc(
  step_count = 198,
  state = MCState(hilbert = Spin(s=1/2, total_sz=0.0, N=8), sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>), n_samples = 1008))
############ step  199
############ param {'Structure Factor': 3.58+0.00j ± 0.10 [

(JsonLog('test', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.022808074951171875
   	Params: 0.0005640983581542969,)

In [25]:
trained_params_list[-1]

{'Structure Factor': 3.55+0.00j ± 0.10 [σ²=10.20, R̂=1.0397],
 'Energy': -13.4007-0.0019j ± 0.0015 [σ²=0.0024, R̂=1.0085]}

In [32]:
trained_params_list

[{'Structure Factor': 1.212+0.000j ± 0.052 [σ²=2.764, R̂=1.0030],
  'Energy': -8.83-0.00j ± 0.18 [σ²=33.02, R̂=1.0051]},
 {'Structure Factor': 1.252+0.000j ± 0.051 [σ²=2.603, R̂=1.0129],
  'Energy': -8.67+0.00j ± 0.18 [σ²=32.73, R̂=1.0200]},
 {'Structure Factor': 1.363+0.000j ± 0.056 [σ²=3.106, R̂=1.0176],
  'Energy': -9.61+0.00j ± 0.17 [σ²=29.17, R̂=1.0092]},
 {'Structure Factor': 1.284+0.000j ± 0.056 [σ²=3.110, R̂=1.0188],
  'Energy': -9.94-0.00j ± 0.16 [σ²=24.65, R̂=1.0097]},
 {'Structure Factor': 1.458+0.000j ± 0.064 [σ²=4.076, R̂=1.0141],
  'Energy': -10.20+0.00j ± 0.15 [σ²=24.02, R̂=1.0132]},
 {'Structure Factor': 1.488+0.000j ± 0.062 [σ²=3.857, R̂=0.9998],
  'Energy': -10.90+0.00j ± 0.14 [σ²=18.81, R̂=1.0045]},
 {'Structure Factor': 1.635+0.000j ± 0.068 [σ²=4.597, R̂=1.0094],
  'Energy': -11.26+0.00j ± 0.13 [σ²=16.17, R̂=1.0043]},
 {'Structure Factor': 1.716+0.000j ± 0.068 [σ²=4.677, R̂=1.0090],
  'Energy': -11.43+0.00j ± 0.12 [σ²=14.86, R̂=1.0105]},
 {'Structure Factor': 1.871+

In [30]:
trained_params = trained_params_list[-1]

# Função para acessar o bias e o kernel após o treinamento
def get_model_params(trained_params):
    kernel = trained_params
    bias = trained_params
    return kernel, bias

In [31]:
# Obtém o kernel e o bias após o treinamento
kernel, bias = get_model_params(trained_params)

print("Kernel shape:", kernel.shape)
print("Bias shape:", bias.shape)

AttributeError: 'dict' object has no attribute 'shape'

In [18]:
gs.state

MCState(
  hilbert = Spin(s=1/2, total_sz=0.0, N=8),
  sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>),
  n_samples = 1008,
  n_discard_per_chain = 100,
  sampler_state = MetropolisSamplerState(# accepted = 20834/20864 (99.85621165644172%), rng state=[1984197433 1949154570]),
  n_parameters = 144)

In [ ]:
trained_params = vs.parameters
# Obtém o kernel e o bias após o treinamento
kernel, bias = get_model_params(trained_params)

In [ ]:
vs.parameters['Dense_0']['bias']

In [ ]:
bias

In [ ]:
# Obtém o kernel e o bias após o treinamento
kernel, bias = get_model_params(trained_params)

print("Kernel shape:", kernel.shape)
print("Bias shape:", bias.shape)

In [ ]:
# Load the data from the .log file
import json

data=json.load(open("test.log"))

iters = data['Energy']['iters']
energy=data['Energy']['Mean']['real']
sf=data['Structure Factor']['Mean']['real']

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(iters, energy, color='blue', label='Energy')
ax1.set_ylabel('Energy')
ax1.set_xlabel('Iteration')
ax2 = ax1.twinx() 
ax2.plot(iters, np.array(sf), color='green', label='Structure Factor')
ax2.set_ylabel('Structure Factor')
ax1.legend(loc=2)
ax2.legend(loc=1)
plt.show()

In [ ]:
print(r"Structure factor = {0:.3f}({1:.3f})".format(np.mean(sf[-50:]),
                                              np.std(np.array(sf[-50:]))/np.sqrt(50)))
print(r"Energy = {0:.3f}({1:.3f})".format(np.mean(energy[-50:]), np.std(energy[-50:])/(np.sqrt(50))))

In [ ]:
E_gs, ket_gs = nk.exact.lanczos_ed(op, compute_eigenvectors=True)
structure_factor_gs = (ket_gs.T.conj()@structure_factor.to_linear_operator()@ket_gs).real[0,0]

In [ ]:
print("Exact Ground-state Structure Factor: {0:.3f}".format(structure_factor_gs))
print("Exact ground state energy = {0:.3f}".format(E_gs[0]))

In [ ]:
# Supondo que você tenha os parâmetros treinados em 'trained_params'
trained_params = model.init(jax.random.PRNGKey(0), jnp.ones((1, 10)))
kernel = model.get_kernel(trained_params)
bias = model.get_bias(trained_params)

print("Kernel shape:", kernel.shape)
print("Bias shape:", bias.shape)

In [ ]:
bias

In [ ]:
kernel